In [4]:
import numpy as np
from aeon.datasets import load_classification
from sklearn.model_selection import train_test_split
import pickle

from models import Models
from morph import Morph

In [10]:
X, y = load_classification("RefrigerationDevices")
print("Shape of X = ", X.shape)
print(f'Classes: {np.unique(y)}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Shape of X =  (750, 1, 720)
Classes: ['1' '2' '3']


In [ ]:
# Train Model
rocket = Models('rocket', X_train, y_train)
rocket.train_rocket()
res = {}

for c in np.unique(y):
    mor = Morph(X_test, y_test, c)
    mor.get_DTWGlobalBorderline(50)
    print(f"Class {c} - vs - Other Classes")
    print("-"*60)
    res[c] = mor.Binay_MorphingCalculater(rocket, verbose=True)

with open('RefrigerationDevices.pkl', 'wb') as f:
    pickle.dump(res, f)


Class 1 - vs - Other Classes
------------------------------------------------------------
Pair-Wise Results:
Pair: (80, 39) -> Morphing percentage: 0.01
Pair: (26, 18) -> Morphing percentage: 0.01
Pair: (69, 2) -> Morphing percentage: 0.01
Pair: (56, 36) -> Morphing percentage: 0.01
Pair: (35, 39) -> Morphing percentage: 0.01
Pair: (69, 9) -> Morphing percentage: 0.01
Pair: (5, 2) -> Morphing percentage: 0.01
Pair: (65, 14) -> Morphing percentage: 0.01
Pair: (30, 9) -> Morphing percentage: 0.01
